### DELAY EDA

In [1]:
# import libraries

!pip install plotly

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px
import plotly.io as pio

import statistics

In [2]:
data = pd.read_excel('get_around_delay_analysis.xlsx')

### STATISTIQUES GLOBALES

In [3]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  float64
 1   car_id                                      21310 non-null  float64
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(5), object(2)
memory usage: 1.1+ MB


In [4]:
display(data.head(4))

display(data.describe(include='all'))


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000.0,363965.0,mobile,canceled,NaN,NaN,NaN
1,507750.0,269550.0,mobile,ended,-81.0,NaN,NaN
2,508131.0,359049.0,connect,ended,70.0,NaN,NaN
3,508865.0,299063.0,connect,canceled,NaN,NaN,NaN


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


In [5]:
# Piechart proporion for state of the cars

df_checkin_type = (data['checkin_type'].value_counts(normalize=True)*100).rename_axis('checkin_type').reset_index(name='counts')

fig = px.pie(df_checkin_type,
             values= 'counts',
             names = 'checkin_type',
             width= 1000,
             title='Proportion of type of reservation'
             )
fig.update_traces(textposition = 'outside', textfont_size = 15)             
fig.update_layout(title_x = 0.5, 
                  margin=dict(l=50,r=50,b=50,t=50,pad=4), 
                  template = 'plotly_dark'
                  )    
fig.show()

In [6]:
# Piechart proporion for state of the cars

data_ratio_state = (data['state'].value_counts(normalize=True)*100).rename_axis('state').reset_index(name='counts')
fig = px.pie(data_ratio_state,
             values='counts',
             names='state', 
             width= 1000,
             title='Proportion of ended versus canceled rentals'
             )
fig.update_traces(textposition = 'outside', textfont_size = 15)             
fig.update_layout(title_x = 0.5, 
                  margin=dict(l=50,r=50,b=50,t=50,pad=4), 
                  template = 'plotly_dark'
                  )    
fig.show()

In [7]:
# histogram for delay at checkout by checkin type WITH PROPORTION OF NUMBER OF RESERVATION
fig = px.histogram(data, x = "state",
                   title = 'Ended - canceled proportion',
                   color = 'checkin_type',
                   barmode ='group',
                   width= 1000,
                   height = 600,
                   histnorm = 'percent',
                   text_auto = True
                  )       
fig.update_traces(textposition = 'outside', textfont_size = 15)
fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  yaxis = {'visible': False}, 
                  xaxis = {'visible': True}, 
                  xaxis_title = '',
                  template = 'plotly_dark'
                  )
fig.update_xaxes(tickfont_size=15)                     
fig.show()

## STATS SUR LES ANNULATIONS

In [8]:
# CANCELATIONS PER CATEGORY

#creation of the categories

data['time_delta_with_previous_rental_in_minutes'] = data['time_delta_with_previous_rental_in_minutes'].fillna('over 12 hours')
#data['time_delta_with_previous_rental_in_minutes'] = data['time_delta_with_previous_rental_in_minutes'].astype(str)


In [9]:
data

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000.0,363965.0,mobile,canceled,NaN,NaN,over 12 hours
1,507750.0,269550.0,mobile,ended,-81.0,NaN,over 12 hours
2,508131.0,359049.0,connect,ended,70.0,NaN,over 12 hours
3,508865.0,299063.0,connect,canceled,NaN,NaN,over 12 hours
4,511440.0,313932.0,mobile,ended,NaN,NaN,over 12 hours
...,...,...,...,...,...,...,...
21305,573446.0,380069.0,mobile,ended,NaN,573429.0,300.0
21306,573790.0,341965.0,mobile,ended,-337.0,NaN,over 12 hours
21307,573791.0,364890.0,mobile,ended,144.0,NaN,over 12 hours
21308,574852.0,362531.0,connect,ended,-76.0,NaN,over 12 hours


In [10]:
df_canceled_temp = data[ data['state'] == 'canceled'].groupby('time_delta_with_previous_rental_in_minutes').count()


In [11]:
# piechart for delay at checkout
fig = px.pie(df_canceled_temp,
             values='state',
             names=df_canceled_temp.index,
             title='Proportion of canceled reservation per category of time delta',
             width= 2000
             )
fig.update_traces(textposition = 'outside', textfont_size = 15)             
fig.update_layout(title_x = 0, 
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  template = 'plotly_dark' )                
fig.show()

On observe que la majeure partie des annulations concerne la catégorie (+12h ou pas de reservation précédente). Les retards de checkout ne sont donc potentiellement responsables que de 7% des annulations au maximum

In [12]:
df_with_previous = data[ data['time_delta_with_previous_rental_in_minutes'] != 'over 12 hours' ]
df_canceled_with_previous = df_with_previous[ df_with_previous['state'] == 'canceled']

df_canceled_with_previous.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 204 to 21272
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   229 non-null    float64
 1   car_id                                      229 non-null    float64
 2   checkin_type                                229 non-null    object 
 3   state                                       229 non-null    object 
 4   delay_at_checkout_in_minutes                0 non-null      float64
 5   previous_ended_rental_id                    229 non-null    float64
 6   time_delta_with_previous_rental_in_minutes  229 non-null    object 
dtypes: float64(4), object(3)
memory usage: 14.3+ KB


In [13]:
df_canceled_ratio = df_canceled_with_previous.groupby('time_delta_with_previous_rental_in_minutes').count()

In [14]:
canceled_mean = df_canceled_with_previous['time_delta_with_previous_rental_in_minutes'].mean()
canceled_median = df_canceled_with_previous['time_delta_with_previous_rental_in_minutes'].median()

In [15]:
# piechart for delay at checkout
fig = px.pie(df_canceled_ratio,
             values = 'state',
             names = df_canceled_ratio.index,
             title='Ratio of canceled reservation by time delta (Under12h only)',
             width= 1000
             )
fig.update_traces(textposition = 'outside', textfont_size = 15)             
fig.update_layout(title_x = 0.5, 
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  template = 'plotly_dark' )                
fig.show()

In [16]:
fig = px.histogram(df_canceled_ratio, x= df_canceled_ratio.index,
                   y = 'state',
                   title = 'Time delta with previous rental in minutes',
                   color = 'checkin_type',
                   barmode = 'group',
                  ) 
fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  xaxis_title = '',
                  template = 'plotly_dark'
                  )   
fig.add_vline(x = canceled_mean , line_width=2 , line_dash = 'dash' , line_color = 'yellow', annotation_text="Mean of values", annotation_position="top right")
fig.add_vline(x = canceled_median , line_width=2 , line_dash = 'dash' , line_color = 'green', annotation_text="Median of values", annotation_position="top right")

fig.update_xaxes(tickfont_size=15,)        
fig.show()

D'aprés ce graphique 50% des annulations avaient un time delta inférieur à 100min et 50% des annulations avaient un time delta inférieur à 300min environ

## STATS SUR LES RETARDS

In [17]:
fig = px.histogram(data, x= 'delay_at_checkout_in_minutes',

                   title = 'delay_at_checkout_in_minutes',
                   color = 'checkin_type',
                   barmode = 'group',
                  ) 
fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  xaxis_title = '',
                  template = 'plotly_dark'
                  )   
fig.update_xaxes(tickfont_size=15,)        
fig.show()

### On va analyser les valeurs en retirant les valeurs extremes
la courbe a des allures de courbe de gauss alors on v a garder toutes les valeurs qui sont dans +/- 2 fois l'ecart type, cela reprensente environ 95% de " l'aire " sous la courbe OU +/- 1 fois qui correspond a 68%

In [18]:
range = data.delay_at_checkout_in_minutes.mean() + data.delay_at_checkout_in_minutes.std()
data_delay_cleaned = data[ (data.delay_at_checkout_in_minutes < range ) & ( data.delay_at_checkout_in_minutes > (-range))]

In [19]:
fig = px.histogram(data_delay_cleaned, x= 'delay_at_checkout_in_minutes',

                   title = 'delay_at_checkout_in_minutes',
                   color = 'checkin_type',
                   barmode = 'group',
                  ) 
fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  xaxis_title = '',
                  template = 'plotly_dark'
                  )   
fig.update_xaxes(tickfont_size=15,)        
fig.show()

In [20]:
cleaned_delay_mean = data_delay_cleaned['delay_at_checkout_in_minutes'].mean()
delay_only_mean = data_delay_cleaned['delay_at_checkout_in_minutes'][ data_delay_cleaned['delay_at_checkout_in_minutes'] > 0].mean()
delay_only_median = data_delay_cleaned['delay_at_checkout_in_minutes'][ data_delay_cleaned['delay_at_checkout_in_minutes'] > 0].median()

print (f'the average delay at checkout is {cleaned_delay_mean} minutes')
print()
print (f'the average delay at checkout FOR DELAYED CHECKOUT ONLY is {delay_only_mean} minutes')
print()
print (f'the median delay at checkout FOR DELAYED CHECKOUT ONLY  is {delay_only_median} minutes')

the average delay at checkout is 16.763710435600125 minutes

the average delay at checkout FOR DELAYED CHECKOUT ONLY is 107.22479683725017 minutes

the median delay at checkout FOR DELAYED CHECKOUT ONLY  is 50.0 minutes


In [25]:
quantile75 = data_delay_cleaned['delay_at_checkout_in_minutes'][ data_delay_cleaned['delay_at_checkout_in_minutes'] > 0].quantile(.7)

df_fig = data_delay_cleaned[ data['delay_at_checkout_in_minutes'] > 0]
fig = px.histogram(df_fig, x = 'delay_at_checkout_in_minutes')

fig.add_vline(x = delay_only_mean , line_width=2 , line_dash = 'dash' , line_color = 'red', annotation_text="Mean of values", annotation_position="top right")
fig.add_vline(x = delay_only_median , line_color = 'black' , annotation_text="Median of values", annotation_position="bottom right")
fig.add_vline(x = quantile75 , line_color = 'green' , annotation_text="70 per cent of delayed checkout", annotation_position="right")

fig.show()

C:\Users\mista\AppData\Local\Temp/ipykernel_2920/3892434470.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



##### A REFAIRE EN PLUS EN BOXPLOT

#### STATS DES DELAY LA OU LA RESERVATION SUIVANTE A ETE ANNULEE

In [26]:
a_list = []

for val in df_canceled_with_previous['previous_ended_rental_id']:
    a_list.append(val)

print(a_list)
print()
print(len(a_list))

[543010.0, 546578.0, 552005.0, 566412.0, 545076.0, 546057.0, 556215.0, 541053.0, 533006.0, 561375.0, 536764.0, 548646.0, 556249.0, 537437.0, 545710.0, 550757.0, 547650.0, 549535.0, 552392.0, 561282.0, 533619.0, 554117.0, 521852.0, 540896.0, 546578.0, 534999.0, 539414.0, 547240.0, 551363.0, 537243.0, 563995.0, 547901.0, 545839.0, 564144.0, 537243.0, 569301.0, 565171.0, 569802.0, 551866.0, 541126.0, 555186.0, 545126.0, 553278.0, 562244.0, 556870.0, 549036.0, 555077.0, 550306.0, 566170.0, 559072.0, 561260.0, 565577.0, 509972.0, 563613.0, 538592.0, 546653.0, 548864.0, 567644.0, 566412.0, 568767.0, 537250.0, 539958.0, 541455.0, 558588.0, 552061.0, 556043.0, 538452.0, 550585.0, 534917.0, 540453.0, 538875.0, 540254.0, 545863.0, 567818.0, 550970.0, 537569.0, 542866.0, 527161.0, 540818.0, 535048.0, 564202.0, 534344.0, 539334.0, 546477.0, 540747.0, 558724.0, 554740.0, 556375.0, 553244.0, 560740.0, 543796.0, 548670.0, 542843.0, 526733.0, 545025.0, 563347.0, 561185.0, 561253.0, 546959.0, 548851.0,

In [27]:
df_shortlist = data[ data['rental_id'].isin(a_list)]
df_shortlist['Late_or_Not'] = df_shortlist['delay_at_checkout_in_minutes'].apply(lambda x: 'Yes' if x > 0 else 'No')




C:\Users\mista\AppData\Local\Temp/ipykernel_2920/3124534823.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
# BFEKOGFPBGGT

df_shortlist_type = (df_shortlist['checkin_type'].value_counts(normalize=True)*100).rename_axis('checkin_type').reset_index(name='counts')

fig = px.pie(df_shortlist_type,
             values= 'counts',
             names = 'checkin_type',
             width= 1000,
             title='Proportion of type of reservation when the next one was canceled'
             )
fig.update_traces(textposition = 'outside', textfont_size = 15)             
fig.update_layout(title_x = 0.5, 
                  margin=dict(l=50,r=50,b=50,t=50,pad=4), 
                  template = 'plotly_dark'
                  )    
fig.show()

In [29]:
display(df_shortlist)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Late_or_Not
122,538569.0,406984.0,mobile,ended,-15.0,NaN,over 12 hours,No
246,546477.0,383309.0,mobile,ended,16.0,548486.0,660.0,Yes
280,548646.0,282893.0,mobile,ended,201.0,NaN,over 12 hours,Yes
417,558088.0,245154.0,connect,ended,-40.0,566170.0,180.0,No
471,562174.0,381499.0,connect,ended,73.0,NaN,over 12 hours,Yes
...,...,...,...,...,...,...,...,...
20839,556043.0,368432.0,mobile,ended,2.0,NaN,over 12 hours,Yes
20877,558858.0,304775.0,connect,ended,18.0,NaN,over 12 hours,Yes
21021,560740.0,383344.0,connect,ended,17.0,534678.0,60.0,Yes
21055,562653.0,341037.0,connect,ended,158.0,NaN,over 12 hours,Yes


In [30]:
#Calculating the percentage of late checkout within the rental id_s where the following reseration has been canceled

Percentage = df_shortlist[ df_shortlist['Late_or_Not'] == 'Yes'].count()[0] / len(df_shortlist) * 100
print('{} % of the reservations within the ones where the next ones have been cancelled had a delayed checkout'.format( round(Percentage, 2)))

47.91 % of the reservations within the ones where the next ones have been cancelled had a delayed checkout


In [31]:
fig = px.histogram(df_shortlist,
                   x = 'delay_at_checkout_in_minutes',
                   color = 'checkin_type',
                   barmode = 'group')

fig.show()

In [32]:
shortlist_delay_mean = df_shortlist['delay_at_checkout_in_minutes'].mean()
shortlist_delay_only_mean = df_shortlist['delay_at_checkout_in_minutes'][ df_shortlist['delay_at_checkout_in_minutes'] > 0].mean()
shortlist_delay_only_median = df_shortlist['delay_at_checkout_in_minutes'][ df_shortlist['delay_at_checkout_in_minutes'] > 0].median()

print (f'the average delay at checkout is {shortlist_delay_mean} minutes')
print()
print (f'the average delay at checkout FOR DELAYED CHECKOUT ONLY is {shortlist_delay_only_mean} minutes')
print()
print (f'the median delay at checkout FOR DELAYED CHECKOUT ONLY  is {shortlist_delay_only_median} minutes')

the average delay at checkout is -32.461139896373055 minutes

the average delay at checkout FOR DELAYED CHECKOUT ONLY is 128.8640776699029 minutes

the median delay at checkout FOR DELAYED CHECKOUT ONLY  is 52.0 minutes


In [33]:
quantile75 = df_shortlist['delay_at_checkout_in_minutes'][ df_shortlist['delay_at_checkout_in_minutes'] > 0].quantile(.7)

df_fig = df_shortlist[ df_shortlist['delay_at_checkout_in_minutes'] > 0]
fig = px.histogram(df_fig, x = 'delay_at_checkout_in_minutes')

fig.add_vline(x = shortlist_delay_only_mean , line_width=2 , line_dash = 'dash' , line_color = 'red', annotation_text= f"Average delay {shortlist_delay_only_mean} min", annotation_position="top right")
fig.add_vline(x = shortlist_delay_only_median , line_color = 'yellow' , annotation_text="Median of values", annotation_position="bottom right")
fig.add_vline(x = quantile75 , line_color = 'green' , annotation_text="70 percent of the delayed checkout", annotation_position="right")


fig.show()